In [1]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric

from  tils import run_experiment
from models import GCN, iterativeGCN
from torch_geometric.datasets import Planetoid

ModuleNotFoundError: No module named 'utils'

In [6]:
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

In [ ]:
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
noise_std = 1

In [ ]:
features = features + torch.normal(mean=0, std=noise_std, size=features.shape)
print("Total data mean: ", torch.mean(features))
print("Mean of feature variances: ", torch.std(features))

In [ ]:
idx_train = range(140)
idx_val = range(200, 500)
idx_test = range(500, 1500)
idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)

In [ ]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
num_epochs = 200
smooth_fac = 0.5

In [ ]:
class ite_GCN(nn.Module):
    def __init__(self, nfeat, nclass, dropout, train_nite, eval_nite=0, allow_grad=True, smooth_fac=0):
        '''     
        - This model is a 1-layer GCN with nite iterations, followed by a linear layer and a log_softmax
            - GC layer:     nfeat to nfeat
            - linear layer: nfeat to nclass, (to cast hidden representations of nodes to a dimension of nclass)
        - Activation: ReLu
        - Input:
            - nfeat:        the number of features of each node
            - nclass:       the number of target classes (we are doing a node classification task here)
            - dropout:      dropout rate
            - train_nite:   the number of iterations during training
            - eval_nite:    the number of iterations during evaluation, 
                            if not specified (or invalid), intialize to the same as train_nite
            - allow_grad:   (bool) defaulted to True. 
                            whether or nor allow gradients to flow through all GC iterations, 
                            if False, gradients will only flow to the last iteration
            - smooth_fac:   a number in [0,1], smoothing factor, controls how much of the OLD iteration result is
                            counted in the skip connection in each iteration
                            for example, smooth_fac = x means y_{i+1} = x * y_i + (1-x) * y_{i+1}
                            Invalid inputs will be treated as 0.
        - Output:
            - A probability vector of length nclass, by log_softmax
        '''
        super(ite_GCN, self).__init__()

        self.gc = GraphConvolution(nfeat, nfeat)
        self.linear_no_bias = nn.Linear(nfeat, nclass, bias=False)
        self.dropout = dropout
        self.train_nite = train_nite
        self.allow_grad = allow_grad
        self.smooth_fac = smooth_fac
        self.eval_nite = eval_nite
        
        if (smooth_fac > 1) or (smooth_fac < 0):
            print("Invalid smoothing factor. Treat as 0.")
            self.smooth_fac = 0
        if (eval_nite <= 0):
            print("Unspecified or invalid number of iterations for inference. Treat as the same as training iterations.")
            self.eval_nite = self.train_nite
        
        print("Initialize a 1-layer GCN with ", self.train_nite, "iterations")
        print("Gradient flows to all iterations: ", allow_grad)

    def run_one_layer(self, x, adj):
        x_old = x
        x_new = self.gc(x, adj)
        x = F.relu(self.smooth_fac * x_old + (1 - self.smooth_fac) * x_new)
        x = F.dropout(x, self.dropout, training=self.training)
        return x

    def forward(self, x, adj):
        if self.training:
            for i in range(self.train_nite):
                if not self.allow_grad:
                    # print("no no no! new new")
                    x = x.detach()
                    x = self.run_one_layer(x, adj)
                else:
                    # print("yea yea yea")
                    x = self.run_one_layer(x, adj)
        else:
            for i in range(self.eval_nite):
                x = self.run_one_layer(x, adj)

        
        x = self.linear_no_bias(x)
        return F.log_softmax(x, dim=1)
        

In [ ]:
class EarlyStopper:
    def __init__(self, model_dict, patience=1, activation_iteration=100):
        self.patience = patience
        self.counter = 0
        self.min_acc_val = -np.inf
        self.saved_model = model_dict
        self.activation_iteration = activation_iteration

    def early_stop(self, acc_val, model_dict, iteration):
        if (acc_val >= self.min_acc_val) or (iteration < self.activation_iteration):
            self.min_acc_val = acc_val
            self.saved_model = model_dict
            self.counter = 0
            print("keep")
        else:
            print("ohohoh")
            self.counter += 1
            if self.counter > self.patience:
                return True
        return False
        
    def get_model(self):
        return self.saved_model



In [ ]:
def run_experiment(num_epochs, model, lr, weight_decay, features, adj, idx_train, idx_val, idx_test, labels, patience=3):
    print("runrunrun!")
    optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)
    t_total = time.time()
    loss_TRAIN = []
    acc_TRAIN = []
    loss_VAL = []
    acc_VAL = []
    # early_stopper = EarlyStopper(model.state_dict().copy(), patience=patience)
    for epoch in range(num_epochs):
        t = time.time()
    
        model.train()
        optimizer.zero_grad()
        
        output = model(features, adj)
        
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        loss_TRAIN.append(loss_train)
        acc_train = accuracy(output[idx_train], labels[idx_train])
        acc_TRAIN.append(acc_train)

        t3 = time.time()
        loss_train.backward()
        t4 = time.time()
        print("backward: ", t4-t3)
        # print("before step: ", model.gc.weight)
        optimizer.step()
        # print("after step: ", model.gc.weight)

        
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        # t1 = time.time()
        output = model(features, adj)
        # print("eval output: ", output)
        # t2 = time.time()
        # print("forward time: ", t2-t1)

        loss_val = F.nll_loss(output[idx_val], labels[idx_val])
        loss_VAL.append(loss_val)
        acc_val = accuracy(output[idx_val], labels[idx_val])
        acc_VAL.append(acc_val)
        print('Epoch: {:04d}'.format(epoch+1),
            'loss_train: {:.4f}'.format(loss_train.item()),
            'acc_train: {:.4f}'.format(acc_train.item()),
            'loss_val: {:.4f}'.format(loss_val.item()),
            'acc_val: {:.4f}'.format(acc_val.item()),
            'time: {:.4f}s'.format(time.time() - t))
        # if early_stopper.early_stop(acc_val=acc_val, model_dict=model.state_dict().copy(), iteration=epoch):
        #     model.load_state_dict(early_stopper.get_model())
        #     break
        

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    # Testing
    test(model, features, adj, idx_test, labels)
    return loss_TRAIN, acc_TRAIN, loss_VAL, acc_VAL

In [ ]:
features.shape[1]

In [ ]:
lr = 0.001
weight_decay = 1e-4
num_epochs=200
model3 = ite_GCN(num_feat=features.shape[1],
            num_class=labels.max().item() + 1,
            dropout=dropout,
            latent_dim=16,
            train_nite= 2,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=smooth_fac)
run_experiment(num_epochs=num_epochs, model=model3, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels, model_name="lalala", run=1)

In [ ]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
num_epochs = 200
smooth_fac = 0.2

In [ ]:
model0 = GCN_5(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout
)
run_experiment(num_epochs=num_epochs, model=model0, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)


# totally messed up

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
l_t = []
for ten in loss_TRAIN:
    l_t.append(ten.item())

In [ ]:
plt.plot(l_t, 'r')

In [ ]:
a_t = []
for ten in acc_TRAIN:
    a_t.append(ten.item())

In [ ]:
plt.plot(a_t, 'r')

In [ ]:
l_v = []
for ten in loss_VAL:
    l_v.append(ten.item())

In [ ]:
plt.plot(l_v, 'r')

In [ ]:
a_v = []
for ten in acc_VAL:
    a_v.append(ten.item())

In [ ]:
plt.plot(a_v, 'r')

In [ ]:
model4 = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=dropout,
            train_nite = 3,
            allow_grad=False,
            smooth_fac=0.3)

In [ ]:
run_experiment(num_epochs=400, model=model4, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels)

In [ ]:
for name, param in model3.named_parameters():
    if param.grad is not None:
        print(name, param.grad.abs().sum())

In [ ]:
model1 = GCN_3(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)

In [ ]:
model2 = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=0,
            train_nite= 2,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=smooth_fac)
run_experiment(num_epochs, model2, lr, weight_decay, features, adj, idx_train, idx_val, idx_test, labels)


In [ ]:
test_losses = []
test_accuracies = []

for i in range(1, 30):
    model = ite_GCN(nfeat=features.shape[1],
            nclass=labels.max().item() + 1,
            dropout=0,
            train_nite= 3,
            eval_nite= i,
            allow_grad=True,
            smooth_fac=smooth_fac)
    model.load_state_dict(model2.state_dict().copy())
    loss_test, acc_test = test(model, features, adj, idx_test, labels)
    test_losses.append(loss_test.item())
    test_accuracies.append(acc_test.item())


In [ ]:
test_losses

In [ ]:
test_accuracies

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test_losses)


In [ ]:
plt.plot(test_accuracies)

In [ ]:
a = np.arange(10)


In [ ]:
fig, ax = plt.subplots(2, 2)
fig.tight_layout(pad=5.0)
ax[0, 0].plot(a, 'b') #row=0, col=0
ax[0, 0].title.set_text("Training loss")
ax[0, 1].plot(a, 'b') #row=0, col=1
ax[0, 1].title.set_text("Training accuracy")
ax[1, 0].plot(a, 'b') #row=1, col=0
ax[1, 0].title.set_text("Validation loss")
ax[1, 1].plot(a, 'b') #row=1, col=1
ax[1, 1].title.set_text("Validation accuracy")
plt.show()

In [ ]:
import os
os.path.dirname('__file__')

In [ ]:
num_runs = 10

In [ ]:
GCN_2_loss = []
GCN_2_acc = []
GCN_2_time = []
for i in range(num_runs):
    # run 2L, 3L ten times each
    model = GCN_2(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)
    loss, acc, train_time = run_experiment(num_epochs=num_epochs, model=model, lr=0.01, 
                   weight_decay=weight_decay, features=features, adj=adj, 
                   idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels, model_name="2L", run=i)
    GCN_2_loss.append(loss)
    GCN_2_acc.append(acc)
    GCN_2_time.append(train_time)
    del model

In [ ]:
from utils import print_stats
print_stats(model_name="Non-iterative 2 layer", acc_test=GCN_2_acc, training_time=GCN_2_time)


In [ ]:
layer1 = GraphConvolution(features.shape[1], hidden)
layer2 = GraphConvolution(features.shape[1], features.shape[1])

In [ ]:
t1 = time.time()
layer1(features, adj)
t2 = time.time()
t3 = time.time()
layer2(features, adj)
t4 = time.time()

In [ ]:
t = t2-t1

In [ ]:
tt = t4-t3

In [ ]:
tt/t

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from utils import load_data, test, train, accuracy, run_experiment
from models import GCN_2, GCN_3, GCN_5, ite_GCN
from layers import GraphConvolution

adj, features, labels = load_data(path="../data/cora/", dataset="cora")
idx_train = range(140)
idx_val = range(200, 500)
idx_test = range(500, 1500)
idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)



In [ ]:
class ite_GCN(nn.Module):
    def __init__(self, num_feat, num_class, dropout, latent_dim,train_nite, eval_nite=0, allow_grad=True, smooth_fac=0):
        '''     
        - The current architecture of the model is:
            - Encoder:      encode features into the latent space
            - GC:           iterations in the latent space
            - Decoder:      decode to the output space
        - Activation: ReLu
        - Input:
            - num_feat:        the number of features of each node
            - num_class:       the number of target classes (we are doing a node classification task here)
            - dropout:      dropout rate
            - latent_dim:   the dimension of the latent space
            - train_nite:   the number of iterations during training
            - eval_nite:    the number of iterations during evaluation, 
                            if not specified (or invalid), intialize to the same as train_nite
            - allow_grad:   (bool) defaulted to True. 
                            whether or nor allow gradients to flow through all GC iterations, 
                            if False, gradients will only flow to the last iteration
            - smooth_fac:   a number in [0,1], smoothing factor, controls how much of the OLD iteration result is
                            counted in the skip connection in each iteration
                            for example, smooth_fac = x means y_{i+1} = x * y_i + (1-x) * y_{i+1}
                            Invalid inputs will be treated as 0.
        - Output:
            - A probability vector of length nclass, by log_softmax
        '''
        super(ite_GCN, self).__init__()

        self.encoder = nn.Linear(num_feat, latent_dim)
        self.gc = GraphConvolution(latent_dim, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_class, bias=False)
        
        self.dropout = dropout
        self.train_nite = train_nite
        self.allow_grad = allow_grad
        self.smooth_fac = smooth_fac
        self.eval_nite = eval_nite
        
        if (smooth_fac > 1) or (smooth_fac < 0):
            # print("Invalid smoothing factor. Treat as 0.")
            self.smooth_fac = 0
        if (eval_nite <= 0):
            # print("Unspecified or invalid number of iterations for inference. Treat as the same as training iterations.")
            self.eval_nite = self.train_nite
        
        print("Initialize a 1-layer GCN with ", self.train_nite, "iterations and latent dimension: ", latent_dim)
        print("Gradient flows to all iterations: ", allow_grad)

    def run_one_layer(self, x, adj):
        x_old = x
        x_new = self.gc(x, adj)
        x = F.relu(self.smooth_fac * x_old + (1 - self.smooth_fac) * x_new)
        x = F.dropout(x, self.dropout, training=self.training)
        return x

    def forward(self, x, adj):
        print("Inpute features shape: ", x.shape)
        print("Adjcency matrix shape: ", adj.shape)
        x = F.relu(self.encoder(x))
        print("Encoded features shape: ", x.shape)
        # x = F.dropout(x, self.dropout, training=self.training)
        
        if self.training:
            num_iterations = self.train_nite
        else:
            num_iterations = self.eval_nite
        
        for i in range(num_iterations):
            x = self.run_one_layer(x, adj)

        print("Feature shape after iterations: ", x.shape)
        x = self.decoder(x)
        print("Decoded shape: ", x.shape)
        return F.log_softmax(x, dim=1)
        

In [ ]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
num_epochs = 200
smooth_fac = 0.5

In [ ]:
model1 = GCN_2(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=dropout)
run_experiment(num_epochs=num_epochs, model=model1, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels, model_name="lalalagua", run=1)

In [ ]:
lr = 0.007
weight_decay = 1e-4
num_epochs = 200
model3 = ite_GCN(num_feat=features.shape[1],
            num_class=labels.max().item() + 1,
            dropout=dropout,
            latent_dim= 16,
            train_nite= 2,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=0.5)
run_experiment(num_epochs=num_epochs, model=model3, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels, model_name="lalala", run=1)

In [ ]:
print(features.shape)
print(labels.shape)


In [ ]:
model = ite_GCN(num_feat=features.shape[1],
            num_class=labels.max().item() + 1,
            dropout=dropout,
            latent_dim= 16,
            train_nite= 2,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=0.5)
aa = model(features, adj)

In [ ]:
import matplotlib.pyplot as plt

class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(1433, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 16)
        )
          
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(16, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1433),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return decoded
    
AE_model = AutoEncoder()
criterion = torch.nn.MSELoss()

AE_epochs = 200
AE_lr = 0.0001
optimizer = torch.optim.Adam(AE_model.parameters(), lr=AE_lr)
AE_train_loss = []

for i in range(AE_epochs):
    output = AE_model(features)
    loss = criterion(output, features)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print("Epoch ", i, ": ", "loss: ", loss.item())
    AE_train_loss.append(loss.item())

plt.plot(AE_train_loss)

In [ ]:
features_new = AE_model.encoder(features).detach()

In [ ]:
features_new.shape

In [ ]:
class ite_GCN_try(nn.Module):
    def __init__(self, num_class, dropout, latent_dim, train_nite, eval_nite=0, allow_grad=True, smooth_fac=0):
        '''     
        - The current architecture of the model is:
           In this one, I separated encoder out of the model
            - GC:           iterations in the latent space
            - Decoder:      decode to the output space
        - Activation: ReLu
        - Input:
            - num_feat:        the number of features of each node
            - num_class:       the number of target classes (we are doing a node classification task here)
            - dropout:      dropout rate
            - latent_dim:   the dimension of the latent space
            - train_nite:   the number of iterations during training
            - eval_nite:    the number of iterations during evaluation, 
                            if not specified (or invalid), intialize to the same as train_nite
            - allow_grad:   (bool) defaulted to True. 
                            whether or nor allow gradients to flow through all GC iterations, 
                            if False, gradients will only flow to the last iteration
            - smooth_fac:   a number in [0,1], smoothing factor, controls how much of the OLD iteration result is
                            counted in the skip connection in each iteration
                            for example, smooth_fac = x means y_{i+1} = x * y_i + (1-x) * y_{i+1}
                            Invalid inputs will be treated as 0.
        - Output:
            - A probability vector of length nclass, by log_softmax
        '''
        super(ite_GCN_try, self).__init__()

        self.gc = GraphConvolution(latent_dim, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_class, bias=False)
        
        self.dropout = dropout
        self.train_nite = train_nite
        self.allow_grad = allow_grad
        self.smooth_fac = smooth_fac
        self.eval_nite = eval_nite
        
        if (smooth_fac > 1) or (smooth_fac < 0):
            print("Invalid smoothing factor. Treat as 0.")
            self.smooth_fac = 0
        if (eval_nite <= 0):
            print("Unspecified or invalid number of iterations for inference. Treat as the same as training iterations.")
            self.eval_nite = self.train_nite
        
        print("Initialize a 1-layer GCN with ", self.train_nite, "iterations")
        print("Gradient flows to all iterations: ", allow_grad)

    def run_one_layer(self, x, adj):
        x_old = x
        x_new = self.gc(x, adj)
        x = F.relu(self.smooth_fac * x_old + (1 - self.smooth_fac) * x_new)
        x = F.dropout(x, self.dropout, training=self.training)
        return x

    def forward(self, x, adj):
        if self.training:
            for i in range(self.train_nite):
                if not self.allow_grad:
                    x = x.detach()
                    x = self.run_one_layer(x, adj)
                else:
                    x = self.run_one_layer(x, adj)
        else:
            for i in range(self.eval_nite):
                x = self.run_one_layer(x, adj)

        x = self.decoder(x)
        return F.log_softmax(x, dim=1)
        

In [ ]:
lr = 0.005
weight_decay = 1e-4
num_epochs = 200
dropout = 0.5
model = ite_GCN_try(
            num_class=labels.max().item() + 1,
            dropout=dropout,
            latent_dim= 16,
            train_nite= 2,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=0.5)
run_experiment(num_epochs=num_epochs, model=model, lr=lr, weight_decay=weight_decay, features=features_new, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels, model_name="lalala", run=1)

In [ ]:
class ite_GCN_try1(nn.Module):
    def __init__(self, num_feat, num_class, dropout, latent_dim,train_nite, eval_nite=0, allow_grad=True, smooth_fac=0):
        '''     
        - The current architecture of the model is:
            - Encoder:      encode features into the latent space
            - GC:           iterations in the latent space
            - Decoder:      decode to the output space
        - Activation: ReLu
        - Input:
            - num_feat:        the number of features of each node
            - num_class:       the number of target classes (we are doing a node classification task here)
            - dropout:      dropout rate
            - latent_dim:   the dimension of the latent space
            - train_nite:   the number of iterations during training
            - eval_nite:    the number of iterations during evaluation, 
                            if not specified (or invalid), intialize to the same as train_nite
            - allow_grad:   (bool) defaulted to True. 
                            whether or nor allow gradients to flow through all GC iterations, 
                            if False, gradients will only flow to the last iteration
            - smooth_fac:   a number in [0,1], smoothing factor, controls how much of the OLD iteration result is
                            counted in the skip connection in each iteration
                            for example, smooth_fac = x means y_{i+1} = x * y_i + (1-x) * y_{i+1}
                            Invalid inputs will be treated as 0.
        - Output:
            - A probability vector of length nclass, by log_softmax
        '''
        super(ite_GCN_try1, self).__init__()

        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(1433, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 16)
        )
        self.gc = GraphConvolution(latent_dim, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_class, bias=False)
        
        self.dropout = dropout
        self.train_nite = train_nite
        self.allow_grad = allow_grad
        self.smooth_fac = smooth_fac
        self.eval_nite = eval_nite
        
        if (smooth_fac > 1) or (smooth_fac < 0):
            print("Invalid smoothing factor. Treat as 0.")
            self.smooth_fac = 0
        if (eval_nite <= 0):
            print("Unspecified or invalid number of iterations for inference. Treat as the same as training iterations.")
            self.eval_nite = self.train_nite
        
        print("Initialize a 1-layer GCN with ", self.train_nite, "iterations")
        print("Gradient flows to all iterations: ", allow_grad)

    def run_one_layer(self, x, adj):
        x_old = x
        x_new = self.gc(x, adj)
        x = F.relu(self.smooth_fac * x_old + (1 - self.smooth_fac) * x_new)
        x = F.dropout(x, self.dropout, training=self.training)
        return x

    def forward(self, x, adj):
        print("Input dimension: ", x.shape)
        x = self.encoder(x)
        print("Encoded dimension: ", x.shape)

        if self.training:
            for i in range(self.train_nite):
                if not self.allow_grad:
                    x = x.detach()
                    x = self.run_one_layer(x, adj)
                else:
                    x = self.run_one_layer(x, adj)
        else:
            for i in range(self.eval_nite):
                x = self.run_one_layer(x, adj)

        x = self.decoder(x)
        return F.log_softmax(x, dim=1)
        

In [ ]:
model = ite_GCN_try1(num_feat=1433,
            num_class=labels.max().item() + 1,
            dropout=dropout,
            latent_dim= 16,
            train_nite= 2,
            eval_nite= 0,
            allow_grad=True,
            smooth_fac=0.5)
run_experiment(num_epochs=num_epochs, model=model, lr=lr, weight_decay=weight_decay, features=features, adj=adj, idx_train=idx_train, idx_val=idx_val, idx_test=idx_test, labels=labels, model_name="lalala", run=1)